In [1]:
%cd ../

/mnt/data/ai4earth/zhangtianning/projects/llm


In [13]:
import json
import numpy as np
import os
import re

import pandas as pd

question_answer = pd.read_excel('/mnt/data/oss_beijing/zhangtianning/LLM/datasets/eval_datasets/RMP12dataset.xlsx')

In [21]:
pairdata = pd.read_json('data/eval/RMP12/pair_question_to_answer_7k.json')

In [24]:
question_index, answer_index_list = pairdata.iloc[9]

In [26]:
truely_question_answer_pair = pd.read_excel('/mnt/data/oss_beijing/zhangtianning/LLM/datasets/eval_datasets/RMP12dataset.xlsx')

In [29]:
question_string = pd.read_csv('data/eval/RMP12/question/question_index_to_string.npy')

In [31]:
question_string.iloc[10]

Question          Who used polarization entangled photon pairs f...
question_index                                                   10
Name: 10, dtype: object

In [34]:
truely_question_answer_pair.iloc[10]

Question    Who used polarization entangled photon pairs f...
Citation                           Clauser and Shimony (1978)
DOI                               10.1088/0034-4885/41/12/002
Name: 10, dtype: object

In [35]:
paper_string = pd.read_csv('data/eval/RMP12/answer_7k/paper_index_of_real_paper.csv')

In [43]:
_map, data = load_wave_data(['data/eval/RMP12/answer_7k/jina_answer_token.npy'])

100%|██████████| 1/1 [00:00<00:00, 61.80it/s]

concat~!
done, cost 7.677078247070312e-05


In [49]:
import transformers
model_name_or_path = 'pretrain_weights/models--jinaai--jina-embeddings-v2-base-en/snapshots/7302ac470bed880590f9344bfeee32ff8722d0e5'
tokenizer = transformers.AutoTokenizer.from_pretrained(
            model_name_or_path,
            model_max_length=128,
            padding_side="right",use_fast=True
        )
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.unk_token


In [ ]:
tokenizer[]

In [52]:
paper_content = pd.read_json('data/eval/RMP12/answer_7k/paper_content_max7000token_paragraph.jsonl',lines=True)

In [60]:
tokens = np.array(tokenizer(paper_content.iloc[147].content,max_length=8192,truncation=True)['input_ids'])

In [66]:
np.abs(tokens[tokens>0] - data[147][data[147]>0]).max()

0

In [54]:
len(paper_string)

423

In [50]:
paper_string.iloc[[147, 148, 149, 150, 151, 152]]

,index,paper_id,sentence_row,arxiv_id,title
147,147,10.1088/0034-4885/41/12/002,0,NaN,Bell{\textquotesingle}s theorem. Experimental ...
148,148,10.1088/0034-4885/41/12/002,1,NaN,Bell{\textquotesingle}s theorem. Experimental ...
149,149,10.1088/0034-4885/41/12/002,2,NaN,Bell{\textquotesingle}s theorem. Experimental ...
150,150,10.1088/0034-4885/41/12/002,3,NaN,Bell{\textquotesingle}s theorem. Experimental ...
151,151,10.1088/0034-4885/41/12/002,4,NaN,Bell{\textquotesingle}s theorem. Experimental ...
152,152,10.1088/0034-4885/41/12/002,5,NaN,Bell{\textquotesingle}s theorem. Experimental ...


In [10]:
# import os
# ROOTPATH='data/unarXive_quantum_physics/quantum_physics/'
# all_json_list = []
# for level_1_name in os.listdir(ROOTPATH):
#     level_1_path = os.path.join(ROOTPATH, level_1_name)
#     for level_2_name in os.listdir(level_1_path):
#         level_2_path = os.path.join(level_1_path, level_2_name)
#         all_json_list.append(level_2_path)

#### Merge jsonl

In [2]:
import json
import pandas as pd
import re
import pathlib
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
from threading import Thread
import argparse

In [21]:
# ### old_way
# import concurrent.futures
# import pandas as pd
# from tqdm.notebook import tqdm
# def process_paper(paper):
#     sections_list = []
#     metadata = paper.metadata
#     name = identify_string_type(metadata['id'])
#     sections = paper.all_text
#     for section_num, sec in enumerate(sections):
#         sections_list.append([name, section_num, sec])
#     return sections_list

# def process_json_file(json_path):
#     with open(json_path) as f:
#         papers = pd.read_json(path_or_buf=f, lines=True)
#     allsections = []
#     for i in range(len(papers)):
#         paper = papers.iloc[i]
#         allsections.extend(process_paper(paper))
#     return allsections

# all_sections = []
# for json_path in all_json_list:
#     all_sections.extend(process_json_file(json_path))
# # with concurrent.futures.ProcessPoolExecutor() as executor:
# #     for sections in tqdm(executor.map(process_json_file, all_json_list), total=len(all_json_list)):
# #         all_sections.extend(sections)




In [5]:
import json
with open("data/unarXive_quantum_physics/quantum_physics/qt_phy_papers_1930_2123.json",'r') as f:
    data = json.load(f)

In [6]:
THEPATH="data/unarXive_quantum_physics/quantum_physics/"
all_json_list = [os.path.join(THEPATH,t) for t in os.listdir(THEPATH) if "qt_phy_papers" in t]

In [27]:
### new_way
import concurrent.futures
import pandas as pd
from tqdm.notebook import tqdm

allsections = []

def process_paper(name,sections ):
    sections_list = []
    name          = identify_string_type(name)
    for section_num, sec in enumerate(sections):
        sections_list.append([name, section_num, sec])
    return sections_list

def process_json_file(json_path):
    with open(json_path,'r') as f:
        papers = json.load(f)
    sections = []
    for name, sentense_list in papers.items():
        sections.extend(process_paper(name,sentense_list))
    return sections

all_sections = []
for json_path in tqdm(all_json_list):
    all_sections.extend(process_json_file(json_path))
# with concurrent.futures.ProcessPoolExecutor() as executor:
#     for sections in tqdm(executor.map(process_json_file, all_json_list), total=len(all_json_list)):
#         all_sections.extend(sections)

  0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
import pandas as pd
import h5py
import multiprocessing as mp
from tqdm.notebook import tqdm

In [19]:
f.close()
hdf5_file = 'data/unarXive_full.clear.abstract.h5'
with h5py.File(hdf5_file, 'w'):pass
f = h5py.File(hdf5_file, 'a')

In [36]:
def write_row_to_hdf5(row):
    #with  as f:
    paper_id,sentence_id,sentence = row
    if str(paper_id) not in f:
        f.create_group(str(paper_id))
    # Store the sentence as a dataset
    f[f'{paper_id}'].create_dataset(str(sentence_id), data=sentence)

In [37]:
for row in tqdm(all_sections):
    write_row_to_hdf5(row)# 

  0%|          | 0/2710382 [00:00<?, ?it/s]

#### 脚本

In [26]:
import json
import pandas as pd
import re
import pathlib
import os
import numpy as np
import matplotlib.pyplot as plt
import tiktoken
import tqdm
from pathlib import Path
from threading import Thread
import argparse
encoding = tiktoken.get_encoding("cl100k_base")
prepositions = {'at', 'in', 'on', 'for', 'with', 'and', 'or', 'nor', 'about', 'as', 'by', 'over', 'according to', 'against', 'along', 'among', 'apart from', 'around', 'as for', 'aside from', 'because of', 'before', 'behind', 'below', 'beneath', 'beside', 'between', 'beyond', 'but', 'by means of', 'concerning', 'despite', 'down', 'due to', 'during', 'except', 'except for', 'in addition to', 'in case of', 'in front of', 'in place of', 'in spite of', 'inside', 'instead of', 'into', 'like', 'near', 'next', 'off', 'onto', 'out', 'out of', 'outside', 'over', 'past', 'since', 'through', 'throughout', 'toward', 'under', 'underneath', 'until', 'up', 'upon', 'with', 'within', 'without'}
prepositions = prepositions|set([t.upper() for t in prepositions])
prepositions = prepositions|set([t[0].upper()+t[1:] for t in prepositions])
prepositions = prepositions|set(['\\.', "\\;",'\\,'])

# format function
def format_text(example_text, insert_data, *args):
    placeholders = re.findall(r"\{\{([^{}]+)\}\}", example_text)

    # Replace the placeholders with corresponding values
    for placeholder in placeholders:
        key = "{{" + placeholder + "}}"
        if placeholder in placeholders:
            example_text = example_text.replace(key, insert_data[placeholder])
    
    return example_text

def replace_continuous_spaces(text):
    return re.sub(r'(?<=\S)\s+', ' ', text)

def replace_start_spaces(text):
    return text#re.sub(r'^\s+', '[SENTENCE_START]', text)


def merge_citations(input_str):
    citation_prefixes = ["DOI:", "ArXiv:", "Ref.", "ALEX:"]
    pattern = r"\[\s*(?:" + "|".join(citation_prefixes) + r")\s*\S+\s*(?:of\s+\S+)?\s*\]"
    # Find all citation marks in the input string
    citations = re.findall(pattern, input_str)
    #print(citations)
    # Replace continuous citation marks with a single mark containing all citations
    output_str = re.sub(pattern + r"(?:\s*,\s*" + pattern + r")+", lambda match: ", ".join(match.group(0).replace('[', '').replace(']', '').split(', ')).join(['[', ']']), input_str)

    return output_str

def replace_number_citation_marks(text, paper_id):
    
    def replace_match(match):
        position, name, citation = match.groups()
        #print(position)
        # Case 1: Beginning of the sentence
        is_start_of_string = match.start() == 0 or text[match.start() - 2] in {'.', '!', '?', ';'}

        if is_start_of_string or position == '':
            return f'[Ref.{citation} of {paper_id}]'
        
        # Case 2: End of the sentence
        if position in prepositions:
            return f'{position} [Ref.{citation} of {paper_id}]'
        
        # Case 3: Middle of the sentence
        return f'(See [Ref.{citation} of {paper_id}])'

    pattern = r'(?:(\b(?:' + '|'.join(prepositions) + r')\b) )?\[([\d, -]+)\]'
    pattern = r'(?:(\b(?:' + '|'.join(prepositions) + r')\b) )?(\w+)?(?:\s)?(?:et al\.\s)?\[([\d, -]+)\]'
    return re.sub(pattern, replace_match, text)


def replace_string_citation_marks(text):
    def replace_match(match):
        position, citation = match.groups()
        is_start_of_string = match.start() == 0 or text[match.start() - 2] in {'.', '!', '?', ';'}

        if is_start_of_string:
            return citation
        # Ignore if the citation is preceded by a preposition
        if position and position.lower() in prepositions:
            return f'{position} {citation}'
        return f'(See {citation})'

    #pattern = r'(?:(\b(?:' + '|'.join(prepositions) + r')\b) )?\[([\d, -]+)\]'
    citation_prefixes = ["DOI:", "ArXiv:", "Ref.", "ALEX:"]
    pattern = r"(?:(\b(?:" + '|'.join(prepositions) + r")\b) )?(\[\s*(?:(?:" + "|".join(citation_prefixes) + r")\s*\S+\s*(?:of\s+\S+)?\s*(?:,\s*)?)+\])".format("|".join(prepositions))
    return re.sub(pattern, replace_match, text)

def format_text_with_values(text, values, type_list,paper_id):
    """
    repalce all text that have the format {{ hash_code }}
    
    """
    pattern = r"(?:Ref\.\s*)?\{\{(" + "|".join(type_list) + r"):([^{}]+)\}\}"
    def replace(match):
        key = "{}:{}".format(match.group(1), match.group(2))
        return values.get(key, key)

    formatted_text = re.sub(pattern, replace, text)
    
    formatted_text = replace_continuous_spaces(formatted_text)
    formatted_text = replace_start_spaces(formatted_text)
    formatted_text = merge_citations(formatted_text)
    formatted_text = replace_string_citation_marks(formatted_text)
    formatted_text = replace_number_citation_marks(formatted_text,paper_id)
    return formatted_text

def split_by_indentation(all_text):
    all_text = "".join(all_text)
    all_text = all_text.replace("\n"," ")
    #all_text = all_text.split('[SENTENCE_START]')
    all_text = re.split(r'\s{2,}', all_text)
    return all_text
# concatenate function
def concatenate_text(all_text, body_text, metadata, encoding=None):  
    
    new_text = split_by_indentation(all_text)
    if len(new_text) == 1:
        # this mean indentation split fail. go back normal split
        pass
    else:
        all_text = new_text
        
    sec_name = None


#     if len(all_text) == 0:
#         return [], []
#     else:
#         # concat all the incompleted sentences
#         all_text, sec_name = concat_by_complete_sentence(all_text, sec_name)
    
    if encoding is None:
        encoding = tiktoken.get_encoding("cl100k_base")
    tokens = [len(encoding.encode(x, disallowed_special=())) for x in all_text]

    #### get the concatenated text lists
    ## do not limit size , we will deal with exceed token problem later
    #cat_text, cat_sec_name = concat_by_token(all_text, sec_name, metadata, tokens)
    cat_text, cat_sec_name = all_text, sec_name
    return cat_text, cat_sec_name

def merge_sentences(sentences):
    merged = []
    for sentence in sentences:
        if merged and sentence[0].islower():
            merged[-1] += ' ' + sentence
        else:
            merged.append(sentence)
    return merged

def concat_by_complete_sentence(sentences, sec_number=None):
    """
     realized function:
         merge this sentence into last sentence if the starting of this sentence is lower alphabeta
    """
    merged = []
    for sentence in sentences:
        if merged and sentence[0].islower():
            merged[-1] += ' ' + sentence
        else:
            merged.append(sentence)
    
    return merged, list(range(len(merged)))   

def concat_by_token(all_text, sec_name, metadata, tokens, min_token = 128, max_token_soft=512):
    # concatenate text to make it
    #cat_text = [metadata.get('abstract', None).replace("\n"," ") or "" ]
    #cat_sec_name = ['abstract']
    cat_text = [] 
    cat_sec_name=[]
    ### we dont need abstract
    
    current_text = ""
    current_sec = sec_name[0]
    current_token = 0
    index = 0
    for i in range(len(all_text)):
        if current_sec != sec_name[i]:
            if current_token < min_token:
                # if too short, append it to the previous paragraph
                cat_text[-1] += current_text
            else:
                cat_sec_name.append(current_sec)
                cat_text.append(current_text)
            current_text  = all_text[i]
            current_token = tokens[i]
            current_sec   = sec_name[i]
            
        if current_sec == sec_name[i] and current_token + tokens[i] <= max_token_soft:
            current_text += "\n"+all_text[i]
            current_token += tokens[i]
            
        else:
            cat_sec_name.append(current_sec)
            cat_text.append(current_text)
            current_text = all_text[i]
            current_token = tokens[i]
            current_sec = sec_name[i]
    
    
    if current_text:
        if current_token < min_token:
            cat_text[-1] += current_text
        else:
            cat_text.append(current_text)
            cat_sec_name.append(current_sec)

    return cat_text, cat_sec_name

import re
def get_name(item_pool):
    if 'ids' not in item_pool:return None
    ids = item_pool['ids']
    if ids['arxiv_id'] !="":
        return f"[ArXiv:{ids['arxiv_id']}]"
    elif ids['doi'] !="":
        return f"[DOI:{ids['doi']}]"
    elif ids['open_alex_id'] !="":
        return f"[ALEX:{ids['open_alex_id'].split('/')[-1]}]"
    else:
        return None

def identify_string_type(s):
    """
    give the unique know of a paper
    """
    pattern1 = r"^\d+\.\d+$"
    pattern2 = r"^[a-zA-Z]+\/\w+$"

    if re.match(pattern1, s):
        return f"ArXiv:{s}"
    elif re.match(pattern2, s):
        return f"ArXiv:{s}"
    else:
        return s

    
def process_a_paper(papers, index=0, encoding=None, use_bib=False):
    """This will change the dataframe `papers` inplace"""
    metadata_i    = papers.loc[index, 'metadata']
    body_text_i   = papers.loc[index, 'body_text']
    bib_entries_i = papers.loc[index, 'bib_entries']
    ref_entries_i = papers.loc[index, 'ref_entries']
    
    # get a dict for format citation/reference data
    insert_data = {}
    type_list = []
    
    # for formula, figure, table
    figure_list =[]
    table_list  =[]
    paper_unique_id  = identify_string_type(metadata_i['id'])
    for id, item in ref_entries_i.items():
        type_list.append(item['type'])
        key = "{}:{}".format(item['type'], id)
        if item['type'] == 'formula':
            value = """${}$""".format(item['latex'])
        elif item['type'] == 'table':
            value = f"""[Table.{len(table_list)} of {paper_unique_id}]"""
            table_list.append(item)
        elif item['type'] == 'figure':
            value = f"""[Figure.{len(table_list)} of {paper_unique_id}]"""
            table_list.append(item)
        else:
            print('='*20)
            print(item)
            print('='*20)
            value = ""
        insert_data[key] = value
        
    # remove duplicate
    type_list = list(set(type_list)) 

    # for citation
    type_list.append('cite')
    
    for i,(_id, item) in enumerate(bib_entries_i.items()):
        key = "{}:{}".format('cite', _id)
        #if use_bib:
        #    value = """[Reference: {}]""".format(item['bib_entry_raw'])
        #else:
        #    value = "[Reference]"
        value = get_name(item)
        value = f"[Ref.{i} of {paper_unique_id}]" if value is None else value
        insert_data[key] = value
    
    all_text = []
    Reference= []
    ReferenceQ= False
    
    for paragraph in body_text_i:
        #print(paragraph.keys())
        text = paragraph['text']
        
        start_string = text.strip()[:20]
        if 'REFERENCES' in start_string or 'Reference' in start_string:
            ReferenceQ=True
            #print(start_string)
        if 'Acknowledgement' in start_string:
            #print(start_string)
            continue
        new_text = format_text_with_values(text, insert_data, type_list,paper_unique_id)
        if len(new_text)==0:
            continue
        
        if all_text and new_text.strip() and (new_text.strip()[0].islower()or new_text.strip()[:5] == 'Proof'):
            all_text[-1]+=' '+new_text.strip()
        elif all_text and all_text[-1].strip() and (all_text[-1].strip()[-1]=="$" or all_text[-1].strip()[-1]==":") and len(all_text[-1].split())<128:
            all_text[-1]+=' '+new_text
        else:
            if ReferenceQ:
                Reference.append(new_text.replace('\n'," "))
            else:
                all_text.append(new_text.replace('\n'," "))
        paragraph['format_text'] = new_text
    
#     for t in all_text:
#         print(len(t.split()))
#         print(t+'\n')
#     raise
    cat_final_i, sec_final_i = concatenate_text(all_text, body_text_i, metadata_i, encoding=encoding)
    Reference = "|.|".join(Reference)
    return all_text, cat_final_i, sec_final_i,Reference

def process_papers(papers, encoding):
    papers['all_text'] = None
    papers['long_text_for_llm'] = None
    papers['long_text_section'] = None
    for i in tqdm.tqdm(range(len(papers)), leave=False, position=1):
        all_text, cat_final_i, sec_final_i,Reference = process_a_paper(papers, i, encoding=encoding)
        papers.at[i, 'all_text'] = all_text
        papers.at[i, 'long_text_for_llm'] = cat_final_i
        papers.at[i, 'long_text_section'] = sec_final_i
        papers.at[i, 'Reference'] = Reference
        
    return papers

def process_and_store(src_path, trg_path, encoding):
    with open(src_path) as f:
        papers = pd.read_json(path_or_buf=f, lines=True)
    papers_new = process_papers(papers, encoding)

    papers_new.to_json(trg_path, orient='records', lines=True)
    
    
def convert_a_file(src_path):
    # trg_path = src_path.replace("unarXive_230324_open_subset", "unarXive_230324_open_subset_new")
    trg_path = src_path.replace("unarXive_230324", "unarXive_clear_20230705")
    trg_dir = os.path.dirname(trg_path)
    pathlib.Path(trg_dir).mkdir(exist_ok=True, parents=True)
    if os.path.exists(trg_path):return
    process_and_store(src_path, trg_path, encoding=encoding)
    
    
def thread_func(all_files, start, end, total_length=5599, show=True, show_iters=5):
    files_done = 0
    with tqdm.tqdm(range(start, end), desc="Main", ncols=100, ascii=True) as pbar:
        for i in pbar:
            file_name = all_files[i]
            convert_a_file(file_name)
            pbar.update(1)
            

In [26]:
!pip install tiktoken

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.2 MB/s eta 0:00:00m eta 0:00:010:00:01


### test

### format abstract

In [1]:
import json

In [5]:
def identify_string_type(s):
    """
    give the unique know of a paper
    """
    pattern1 = r"^\d+\.\d+$"
    pattern2 = r"^[a-zA-Z]+\/\w+$"

    if re.match(pattern1, s):
        return f"ArXiv:{s}"
    elif re.match(pattern2, s):
        return f"ArXiv:{s}"
    else:
        return s
def read_jsonl(path):
    try:
        with open(path,'r') as f:
            data = [json.load(f)]
    except json.JSONDecodeError:
        with open(path,'r') as f:
            data = []
            for line in f:
                data.append(json.loads(line))
    return data

def structure_dict(_dict):
    paper_id = _dict['paper_id']
    abstract = _dict['abstract']
    key_words= _dict['question']
    pattern = r'\d+\.\s*(.*?)(?=\s*\d+\.\s*|$)'
    words   = re.findall(pattern, key_words)
    if len(words)<1:
        words = key_words.split(',')
    return [paper_id] + [abstract] + words[:10]
import os
ROOTPATH='data/unarXive/split'
all_json_list = []
for level_1_name in os.listdir(ROOTPATH):
    
    level_1_path = os.path.join(ROOTPATH, level_1_name)
    for level_2_name in os.listdir(level_1_path):
        level_2_path = os.path.join(level_1_path, level_2_name)
        all_json_list.append(level_2_path)

In [6]:
import concurrent.futures
from tqdm.notebook import tqdm

def read_json_files(path):
    return read_jsonl(path)


all_data = []

with concurrent.futures.ProcessPoolExecutor(16) as executor:
    results = list(tqdm(executor.map(read_json_files, all_json_list), total=len(all_json_list)))

for result in results:
    all_data += result

  0%|          | 0/5599 [00:00<?, ?it/s]

In [11]:
dois = []
for i, d in enumerate(all_data):
    dois.append([i,d['metadata']['doi']])
    

In [15]:
dois = [[a,b] for a,b in dois if b is not None]

In [20]:
doismap = dict([[b,a] for a,b in dois])

In [19]:
import pandas as pd
eval_data = pd.read_csv("data/eval/RMP12.csv", delimiter='\t')

In [22]:
for doi in eval_data['DOI']:
    if doi in doismap:
        print(doismap[doi])

In [19]:
import h5py

In [ ]:
# Create an HDF5 file and store the data


In [22]:
import re

In [ ]:
hdf5_file = 

In [34]:
with h5py.File('dataset/unarXive.clear.title.h5', 'w') as f:
    for row in tqdm(all_data):
        paper_id    = 'title'
        sentence_id = identify_string_type(row['metadata']['id'])
        title    = row['metadata']['title'].replace('\n', " ").replace("  ", " ").replace("  ", " ").strip()
        
        # Create a group for each paper_id if it doesn't exist
        if str(paper_id) not in f:
            f.create_group(str(paper_id))

        # Store the sentence as a dataset
        f[f'{paper_id}'].create_dataset(str(sentence_id), data=title)

  0%|          | 0/165000 [00:00<?, ?it/s]

### format main content

In [12]:
import os
ROOTPATH='dataset/unarXive_clear_20230705/'
all_json_list = []
for level_1_name in os.listdir(ROOTPATH):
    level_1_path = os.path.join(ROOTPATH, level_1_name)
    for level_2_name in os.listdir(level_1_path):
        level_2_path = os.path.join(level_1_path, level_2_name)
        all_json_list.append(level_2_path)
with open(all_json_list[73]) as f:
    papers = pd.read_json(path_or_buf=f, lines=True)
index    = 0
result   = process_a_paper(papers,index)
metadata = papers.iloc[index].metadata
title    =  metadata['title'].replace('\n',' ')
print(f"""===========================
            {title} 
===========================""")
print(metadata['abstract'].strip().replace('\n'," "))
print("="*20)
for t in result[0]:
    print(t+'\n')

            Connecting The Dots To Combat Collective Fraud 
Modern fraudsters write malicious programs to coordinate a group of accounts to commit collective fraud for illegal profits in online platforms. These programs have access to a set of finite resources - a set of IPs, devices, and accounts etc. and sometime manipulate fake accounts to collaboratively attack the target system. Inspired by these observations, we share our experience in building two real-time risk control systems to detect collective fraud. We show that with TigerGraph, a powerful graph database, and its innovative query language - GSQL, data scientists and fraud experts can conveniently implement and deploy an end-to-end risk control system as a graph database application.
Detecting fraudulent activity is a never ending battle in the digital world. More and more merchants and financial organizations are targets for fraudsters and cybercriminals. Merchants and financial services organizations will spend $9.3 billi

In [15]:
len(df)

10727894

In [ ]:
def write_row_to_hdf5(row, hdf5_file):
    with h5py.File(hdf5_file, 'a') as f:
        paper_id    = row['paper_id']
        sentence_id = row['section_num']
        sentence    = row['section']
        
        # Create a group for each paper_id if it doesn't exist
        if str(paper_id) not in f:
            f.create_group(str(paper_id))

        # Store the sentence as a dataset
        f[f'{paper_id}'].create_dataset(str(sentence_id), data=sentence)

# Create an HDF5 file and store the data
hdf5_file = 'dataset/unarXive.clear.sections.h5'

# Initialize the HDF5 file
with h5py.File(hdf5_file, 'w'):
    pass

# Use multiprocessing to parallelize the writing process

In [90]:
data = read_jsonl(all_json_list[9])

In [89]:
def read_jsonl(path):
    try:
        with open(path,'r') as f:
            data = [json.load(f)]
    except json.JSONDecodeError:
        with open(path,'r') as f:
            data = []
            for line in f:
                data.append(json.loads(line))
    return data

def structure_dict(_dict):
    paper_id = _dict['paper_id']
    abstract = _dict['abstract']
    key_words= _dict['question']
    pattern = r'\d+\.\s*(.*?)(?=\s*\d+\.\s*|$)'
    words   = re.findall(pattern, key_words)
    if len(words)<1:
        words = key_words.split(',')
    return [paper_id] + [abstract] + words[:10]

In [18]:
from tqdm.notebook import tqdm

In [19]:
all_data = [] 
for path in tqdm(all_json_list):
    all_data+=read_jsonl(path)

  0%|          | 0/5599 [00:00<?, ?it/s]

In [93]:
for t in all_data[0]['bib_entries']:
    print(t)

9209c4bb343e88ab805ed0007149eddc0f2f842c
09f7d8e302939677061e718778e2ebf9e19fdb22
b62c466cacc990508dbbb34e5c15681be91cf252
9003f7c17f3de907d848f52ec6ff19fc43cb69c5
9555e14b8722bde3a8379bb02f6f936074b29748
2b4b0671764aa55c2b99d86f2c03d69f75f0a39c
5c6192d1af2a4eebb5b6f7f67b75c8edb799c749
487d1c1bcc95f533a0b46616aaf3f4acc954db7b
04dc09d45391dd39dc88899fa1416ac33387e9de
d60fe6060c930c93a875dddae6d75e43434d1fee
948f6a6ab7a7048c510baf1914e6791f2fdbbd2d
ef5442955c843be18174385155cf6fde5535c7e1
44a90f86c67019fefd4b5f779c742f2e719739d4
59dfdf31f249c50692d92c63b95423ccd0275b18
6f93b75b5b4b84abfaee5ff2fa4804904ab2e7da
c0d112191c8d160f7260384e5f8f24238d934202
1bb8f31715f1a294eece336bee15d868f2f55f04


In [68]:
np.where(aaa==aaa.min())

(array([493]),)

In [11]:
allquesion = np.array([structure_dict(d) for d in  all_data])

In [13]:
allquesion

array([], dtype=float64)

In [135]:
import pandas as pd
import numpy as np

In [164]:
rowstats = [len(t) for t in allquesion]

In [173]:
badslots = np.where(np.array(rowstats)<12)[0]

In [178]:
good_data = allquesion[np.array(rowstats)==12]

In [182]:
column_titles = ['paper_id', 'abstract', 'key_word_1', 'key_word_2', 'key_word_3',
                 'key_word_4', 'key_word_5', 'key_word_6', 'key_word_7', 'key_word_8',
                 'key_word_9', 'key_word_10']
df = pd.DataFrame(good_data.tolist(), columns=column_titles)

In [184]:
df.to_csv('unArXiv.key_word_from_abstract.csv')